In [219]:
import bs4
import requests
import html5lib
import time
import json

In [220]:
base_url = "http://www.ilga.gov"

In [221]:
leg_url = base_url + "/legislation/"

In [182]:
print(leg_url)

http://www.ilga.gov/legislation/


In [222]:
r = requests.get(leg_url)

In [223]:
html_text = r.text.encode('iso-8859-1')

In [224]:
soup = bs4.BeautifulSoup(html_text, "html5lib")

In [225]:
links = soup.find_all("a")

In [226]:
mid_links = []
for a in links:
    if a.has_attr("href") and a['href'].startswith('grplist.asp'):
        mid_url = leg_url + a['href']
        mid_r = requests.get(mid_url)
        mid_html_text = mid_r.text.encode('iso-8859-1')
        mid_soup = bs4.BeautifulSoup(mid_html_text, "html5lib")
        mid_links.extend(mid_soup.find_all("a"))
        

In [227]:
bill_links = []
for mid_a in mid_links:
    if mid_a.text.startswith("SB") or mid_a.text.startswith("HB"):
        bill_url = base_url + mid_a['href']
        bill_links.append(bill_url)

In [228]:
len(bill_links)

9734

In [207]:
r_test = requests.get(bill_links[1])
html_test = r_test.text.encode('iso-8859-1')
soup_test = bs4.BeautifulSoup(html_test, "html5lib")

In [208]:
soup_test

<html lang="en"><!-- Trigger/Open The Modal --><head></head><body alink="#9933FF" bgcolor="#FFFFFF" leftmargin="0" link="#3366FF" onload="document.NumString.NumSearch.focus();" text="#000000" topmargin="0" vlink="#663366"><div style="position: fixed; z-index: 999; top: 5; left: 600; background-color: navy; display: block">
<button id="myBtn" style="color: white; background-color: navy; display: block">Translate Website</button></div>
<!-- The Modal -->
<div class="modal" id="myModal" style="display: none">
  <!-- Modal content -->
  <div class="modal-content">
      <div class="modal-header"><h3>
    <span class="close">×</span></h3></div>    
    <p>The Illinois General Assembly offers the Google Translate™ service for visitor convenience. In no way should it be considered accurate as to the translation of any content herein.</p>
    <p>Visitors of the Illinois General Assembly website are encouraged to use other translation services available on the internet.</p>
    <p>The English l

In [209]:
#bill_info = {}
soup_test.find_all("span")

[<span class="close">×</span>,
 <span style="white-space: nowrap"><a class="goog-logo-link" href="https://translate.google.com" target="_blank"><img alt="Google Translate" height="14" src="https://www.gstatic.com/images/branding/googlelogo/1x/googlelogo_color_42x16dp.png" style="padding-right: 3px;" width="37"/>Translate</a></span>,
 <span class="heading">Bill Status of SB0002</span>,
 <span class="content"><i>101st General Assembly</i></span>,
 <span class="heading2">Short Description:</span>,
 <span class="content">INFRASTRUCTURE-TECH</span>,
 <span class="heading2">Senate Sponsors</span>,
 <span class="content">Sen. </span>,
 <span class="heading2">Last Action</span>,
 <span class="heading2">Statutes Amended In Order of Appearance</span>,
 <span class="heading2">Synopsis As Introduced</span>,
 <span class="content notranslate">     Creates the Illinois Infrastructure Act of 2019. Contains only a short title provision.</span>,
 <span class="heading2">Actions</span>]

In [210]:
heading_tags = soup_test.find_all('span', class_ = "heading2")
for tag in heading_tags:
    if tag.text.startswith("Actions"):
        actions_tag = tag.next_sibling.next_sibling.next_sibling

In [211]:
actions_tag

<table border="1" bordercolor="black" cellpadding="2" cellspacing="0" width="600"><tbody><tr><td align="center" bgcolor="navy" class="whiteheading" width="13%">Date</td><td align="center" bgcolor="navy" class="whiteheading" width="12%">Chamber</td><td align="left" bgcolor="navy" class="whiteheading" width="75%"> Action</td></tr><tr><td align="right" class="content" valign="top" width="13%">  1/9/2019</td><td align="center" class="content" valign="top" width="12%">Senate</td><td align="left" class="content" valign="top" width="75%">Filed with Secretary by <a href="../../senate/senator.asp?GA=101&amp;MemberID=2534">Sen. John J. Cullerton</a></td></tr><tr><td align="right" class="content" valign="top" width="13%">  <b>1/9/2019</b></td><td align="center" class="content" valign="top" width="12%"><b>Senate</b></td><td align="left" class="content" valign="top" width="75%"><b>First Reading</b></td></tr><tr><td align="right" class="content" valign="top" width="13%">  1/9/2019</td><td align="cen

In [195]:
table_tags

[<td align="left" bgcolor="navy" class="whiteheading" width="75%"> Action</td>,
 <td align="left" class="content" valign="top" width="75%">Filed with Secretary by <a href="../../senate/senator.asp?GA=101&amp;MemberID=2535">Sen. Kimberly A. Lightford</a></td>,
 <td align="left" class="content" valign="top" width="75%"><b>First Reading</b></td>,
 <td align="left" class="content" valign="top" width="75%">Referred to <a href="../../senate/committees/members.asp?GA=101&amp;committeeID=2341">Assignments</a></td>,
 <td align="left" class="content" valign="top" width="75%">Approved for Consideration <a href="../../senate/committees/members.asp?GA=101&amp;committeeID=2341">Assignments</a></td>,
 <td align="left" class="content" valign="top" width="75%">Placed on Calendar Order of 2nd Reading January 29, 2019</td>,
 <td align="left" class="content" valign="top" width="75%"><b>Second Reading</b></td>,
 <td align="left" class="content" valign="top" width="75%">Placed on Calendar Order of 3rd Readi

In [196]:
table_tags = actions_tag.find_all('td', align = "left")
for i, tag in enumerate(table_tags):
    if tag.text.startswith("Assigned to"): 
        committee_tag = tag
        committee = tag.a.text
        break
print(committee_tag)
print(committee)
        

<td align="left" class="content" valign="top" width="75%">Assigned to <a href="../../house/committees/members.asp?GA=101&amp;committeeID=2309">Labor &amp; Commerce Committee</a></td>
Labor & Commerce Committee


Bill Number

In [197]:
bill_number = soup_test.title.text.split()[-1]
bill_info[bill_number] = {}
print(bill_info)

{'SB0001': {}}


Bill Title (Short Description) and Status

In [198]:
heading_tags = soup_test.find_all('span', class_ = "heading2")
for tag in heading_tags:
    if tag.text.startswith("Short Description"):
        description_tag = tag
    if tag.text.startswith("Last Action"):
        last_action_marker = tag.next.next.next
        break

In [199]:
bill_short_descr = description_tag.next.next.next.text
bill_info[bill_number]['short description'] = bill_short_descr
print(bill_info)

{'SB0001': {'short description': 'MINIMUM WAGE/INCOME TAX CREDIT'}}


In [200]:
last_action_tags = last_action_marker.find_all('td', class_ = "content")
for tag in last_action_tags:
    if tag['align'] == 'left': last_action = tag.text
    if tag['align'] == 'right': last_action_date = tag.text.strip()
    if tag['align'] == 'center': last_action_body = tag.text

In [201]:
bill_info[bill_number]['last action'] = {'action': last_action, 
                                         'date': last_action_date, 
                                         'body': last_action_body}
if last_action.endswith('Assignments'): bill_status = 'In process - not yet assigned to committee'
elif last_action.startswith("Public Act"): bill_status = 'Passed and signed into law'
else: bill_status = 'In process'

Senate Sponsors

In [ ]:
next_tag = None
for tag in heading_tags:
    if tag.text.startswith("Senate Sponsors"):
        next_tag = tag.next_sibling.next_sibling
        break
    

In [ ]:
senate_sponsors = []
while True:
    if next_tag is None: break
    if isinstance(next_tag, bs4.element.Tag) and next_tag.has_attr("href"):
        senate_sponsors.append(next_tag.text)
    next_tag = next_tag.next_sibling
    if next_tag in heading_tags: break

In [ ]:
bill_info[bill_number]['senate sponsors'] = senate_sponsors
print(bill_info)

House Sponsors

In [ ]:
next_tag = None
for tag in heading_tags:
    if tag.text.startswith("House Sponsors"):
        next_tag = tag.next_sibling.next_sibling
        break 

In [ ]:
house_sponsors = []
while True:
    if next_tag is None: break
    if isinstance(next_tag, bs4.element.Tag) and next_tag.has_attr("href"):
        house_sponsors.append(next_tag.text)
    next_tag = next_tag.next_sibling
    if next_tag in heading_tags: break

In [ ]:
bill_info[bill_number]['house sponsors'] = house_sponsors
print(bill_info)

Synopsis

In [ ]:
synopsis_list = []
for tag in soup_test.find_all('span', class_ = "content notranslate"):
    synopsis_list.append(tag.text.strip())
synopsis = "  ".join(synopsis_list)  

In [ ]:
bill_info[bill_number]['synopsis'] = synopsis.strip()
print(bill_info)

In [212]:
def add_bill_info(link, bill_info):
    r_bill = requests.get(link)
    html_bill = r_bill.text.encode('iso-8859-1')
    soup_bill = bs4.BeautifulSoup(html_bill, "html5lib")
    
    #Add bill number
    bill_number = soup_bill.title.text.split()[-1]
    bill_info[bill_number] = {}
    
    #Bill description and last action
    heading_tags = soup_bill.find_all('span', class_ = "heading2")
    for tag in heading_tags:
        if tag.text.startswith("Short Description"):
            description_tag = tag
        if tag.text.startswith("Last Action"):
            last_action_marker = tag.next.next.next
            break
    bill_short_descr = description_tag.next.next.next.text
    bill_info[bill_number]['short description'] = bill_short_descr
    last_action_tags = last_action_marker.find_all('td', class_ = "content")
    for tag in last_action_tags:
        if tag['align'] == 'left': last_action = tag.text
        if tag['align'] == 'right': last_action_date = tag.text.strip()
        if tag['align'] == 'center': last_action_body = tag.text
    bill_info[bill_number]['last action'] = {'action': last_action, 
                                             'date': last_action_date, 
                                             'body': last_action_body}
    if last_action == 'Referred to Assignments': bill_status = 'In process - not yet assigned to committee'
    elif last_action.startswith("Public Act"): bill_status = 'Passed and signed into law'
    else: bill_status = 'In process'
    bill_info[bill_number]['status'] = bill_status
    
    #Committee
    committee = None
    if bill_status is not 'In process - not yet assigned to committee':
        for tag in heading_tags:
            if tag.text.startswith("Actions"):
                actions_tag = tag.next_sibling.next_sibling.next_sibling
        table_tags = actions_tag.find_all('td', align = "left")
        for i, tag in enumerate(table_tags):
            if tag.text.startswith("Assigned to"): 
                committee_tag = tag
                committee = tag.a.text
                break
    bill_info[bill_number]['committee'] = committee
    
    #Senate sponsors
    next_tag = None
    for tag in heading_tags:
        if tag.text.startswith("Senate Sponsors"):
            next_tag = tag.next_sibling.next_sibling
            break
    senate_sponsors = []
    while True:
        if next_tag is None: break
        if isinstance(next_tag, bs4.element.Tag) and next_tag.has_attr("href"):
            senate_sponsors.append(next_tag.text)
        next_tag = next_tag.next_sibling
        if next_tag in heading_tags: break
    bill_info[bill_number]['senate sponsors'] = senate_sponsors
    
    #House sponsors
    next_tag = None
    for tag in heading_tags:
        if tag.text.startswith("House Sponsors"):
            next_tag = tag.next_sibling.next_sibling
            break
    house_sponsors = []
    while True:
        if next_tag is None: break
        if isinstance(next_tag, bs4.element.Tag) and next_tag.has_attr("href"):
            house_sponsors.append(next_tag.text)
        next_tag = next_tag.next_sibling
        if next_tag in heading_tags: break
    bill_info[bill_number]['house sponsors'] = house_sponsors
    
    #Synopsis
    synopsis_list = []
    for tag in soup_bill.find_all('span', class_ = "content notranslate"):
        synopsis_list.append(tag.text.strip())
    synopsis = "  ".join(synopsis_list)
    bill_info[bill_number]['synopsis'] = synopsis.strip()

In [231]:
bill_info = {}

In [232]:
t0 = time.time()
for link in bill_links:
    add_bill_info(link, bill_info)        
delta_t = time.time() - t0  
print(delta_t)

2843.1397190093994


Export the bill info to a json file

In [233]:
with open('bill_info.json', 'w') as f:
    json.dump(bill_info, f)

### Make list of keywords by committee

In [234]:
bill_keywords = set()
committee_keywords = {}
for bill_dict in bill_info.values():
    keyword = bill_dict['short description']
    committee = bill_dict['committee']
    bill_keywords.add(keyword)
    if committee not in committee_keywords: committee_keywords[committee] = set()
    committee_keywords[committee].add(keyword)

In [238]:
for com, key in committee_keywords.items():
    committee_keywords[com] = list(key)

In [239]:
committee_keywords

{'Labor & Commerce Committee': ['TRAMPOLINE COURTS-SAFETY',
  'MINIMUM WAGE-WORK STUDY/INTERN',
  'VEH CD-LICENSE-SUSPENSION',
  'WAGES PENALTY FAIL TO PAY',
  'LABOR-FIRE FIGHTER SUPERVISOR',
  'FAMILY AND MEDICAL LEAVE ACT',
  'SCHOOL CONFERENCE NO TERMINATE',
  'WORKERS COMP-FEE SCHEDULE',
  'COMPASSIONATE USE-LIABILITY',
  'POLYGRAPH EXAMS',
  'WORKERS COMP-CAUSATION',
  'BUS CORP-BOARD RACE SEX',
  'ED LABOR REL-CONTRACT APPROVAL',
  'MINIMUM WAGE-TAX-EXEMPT ORG',
  'EQUAL PAY ACT-WAGE HISTORY',
  'WORKER COMP-DRUG FORMULARY',
  'COUNTIES-ADULT ENTERTAINMENT',
  'WAGE THEFT-DAMAGES AMOUNT',
  'POLICE DISCIPLINARY COMPLAINT',
  'KENNEL OPERATION-FIRE ALARMS',
  'FIRE FIGHTER TRAINING',
  'ATTY FEES WORKER COMP 15% MAX',
  'PREVAILING WAGE INFRASTRUCTURE',
  'GOVERNMENT-TECH',
  'FRANCHISEE COUNSEL REQUIREMENT',
  'UNEMP INS-REASONABLE ASSURANCE',
  'WORKERS COMP-BENEFIT RATES',
  'ED EMPLOYEE-IMPASSE PROCEDURES',
  'INDEPENDENT CONTRACTOR PAYMENT',
  'HIGH IMPACT BUSINESS-WIND',
  

In [236]:
len(bill_keywords)

4890

In [240]:
with open('committee_keywords.json', 'w') as f:
    json.dump(committee_keywords, f)